# Section 3: Python & Hugging Face

## Load the Dataset

In [ ]:
import pandas as pd

df = pd.read_csv('../data/traininig-dataset.csv')

print(f"Dataset loaded: {len(df)} samples")

print(f"\nLabel distribution:")
label_counts = df['label'].value_counts().reset_index()
label_counts.columns = ['label', 'count']
print(label_counts.to_string(index=False))

print(f"\nFirst few rows:")
print(df.head(3).to_string(index=False))

## Tokenize the Text Field

### Split into Train and Validation Sets

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

print(f"\nTrain: {len(train_df)} samples | Validation: {len(val_df)} samples")

### Load Tokenizer

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

### Tokenization Function

In [ ]:
from datasets import Dataset

def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

### Convert to Hugging Face Datasets and Tokenize

In [ ]:
train_dataset = Dataset.from_pandas(train_df[['text', 'label']]).map(tokenize_function, batched=True)
val_dataset = Dataset.from_pandas(val_df[['text', 'label']]).map(tokenize_function, batched=True)

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

print("\nTokenization complete!")

## Train a Small Classifier

### Load Model

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

### Training Configuration

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="epoch", 
    logging_steps=5,
    seed=42
)

### Metrics Function

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions)
    return {'accuracy': accuracy, 'f1': f1}

### Initialize Trainer

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

### Train the Model

In [ ]:
print("\nStarting training (2 epochs)...")
trainer.train()
print("Training complete!")

### Report Validation Accuracy and F1 Score

In [ ]:
eval_results = trainer.evaluate()

print("\n" + "="*50)
print("VALIDATION RESULTS")
print("="*50)
print(f"Accuracy: {eval_results['eval_accuracy']:.4f} ({eval_results['eval_accuracy']*100:.2f}%)")
print(f"F1 Score: {eval_results['eval_f1']:.4f}")
print("="*50)

# Section 4: Confusion Matrix

## Generate Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

predictions = trainer.predict(val_dataset)
pred_labels = np.argmax(predictions.predictions, axis=-1)
true_labels = val_df['label'].values

cm = confusion_matrix(true_labels, pred_labels)

## Plot Confusion Matrix

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['No HMB', 'HMB'],
            yticklabels=['No HMB', 'HMB'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - HMB Classification')
plt.tight_layout()

print("\nDone!")